In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import random
from __future__ import print_function
import pickle
import pandas as pd
import sqlite3

In [ ]:
# Search for all of the possible permutations of data science jobs
# Pull from Indeed.com

search_list = ['data+analyst','business+analyst','data+scientist','data+engineer','data+architect']

job_ids = []

for title in search_list:
    
    test = "https://www.indeed.com.sg/jobs?q={}&l=Singapore&jt=fulltime&start=".format(title)
        
    sleep(random.randint(10, 20))
    
    for page in range(0,200,10): 
        
        #print('Parsing page {}'.format(page))
        
        url = test + '{}'.format(page)
        
        # make request for that page
        sauce = requests.get(url).text
    
        # turn into a BeautifulSoup object
        soup = BeautifulSoup(sauce, 'lxml')
    
        # get the job IDs
        for listing in soup.find_all('a', {'href': re.compile(r'jk\=(\w+)')}):
            job_id = re.findall(r'jk\=(\w+)', listing['href'])[0]
            job_ids.append(job_id)
    
        sleep(random.randint(3, 15))
        
        print('{}, '.format(page), end='')
    print(title.replace('+',' '), end='')

In [ ]:
# Remove all duplicates
job_ids = list(set(job_ids))

In [ ]:
len(job_ids)

In [ ]:
# Pickle the job ids
# pickle.dump(job_ids, open('job_ids_1604.p','wb'))

In [ ]:
# Unpickle the job ids
# pickle_off = open('job_ids.p','rb')
# job_ids = pickle.load(pickle_off)

In [ ]:
titles = []
company_name = []
job_descriptions = []

no_data = 'NA'

for id in job_ids:
    
    url = "https://www.indeed.com.sg/viewjob?jk={}".format(id)
    
    sauce = requests.get(url).text
    soup = BeautifulSoup(sauce, 'lxml')
    
    print('--', end='')
    
    sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism

    for i in soup.find_all('table', {'id': 'job-content'}):
        
        # Extract Job Titles
        try:
            job_title = i.find('b', {'class': 'jobtitle'}).font.text
            titles.append(job_title)
        except Exception:
            title.append(no_data)
        
        # Extract Company Name
        try:
            company = i.find('span', {'class': 'company'}).text
            company_name.append(company)
        except Exception:
            company_name.append(no_data)
    
        # Extract Job Description
        try:
            job_description = i.find('span', {'class':'summary'}).text.strip()
            job_descriptions.append(job_description)
        except Exception:
            job_descriptions.append(no_data)
    
        sleep(random.randint(5, 15)) # Avoiding the anti-bot mechanism
    
    print('>', end='')

In [ ]:
print('Total number of items found: {}'.format(len(titles)))

In [ ]:
print('Total number of items found: {}'.format(len(company_name)))

In [ ]:
print('Total number of items found: {}'.format(len(job_descriptions)))

In [ ]:
# # Unpickle previously scraped data
# pickle_off = open('job_i1ds.p','rb')
# old_job_ids = pickle.load(pickle_off)

In [ ]:
# DataFrame
df = pd.DataFrame({'id': job_ids,'title': titles,'description': job_descriptions})

In [ ]:
df.head()

In [ ]:
df['id'].duplicated(keep='first').sum()

In [ ]:
df.drop_duplicates(subset='id', keep='first', inplace=True)

In [ ]:
# # Check against already mined IDs
# mined_ids = pickle.load(open('job_ids.p','rb'))

In [ ]:
# # Check against already scraped data
#
# df = df[~df['id'].isin(mined_ids)]

In [ ]:
# Saving to csv instead
# df.to_csv('scraped_0804.csv', encoding='utf-8')

In [ ]:
# # Saving to a SQL file
# connection = sqlite3.connect('job_scraped.db.sqlite')
# df.to_sql(name = 'jobs', con = connection, if_exists = 'append', index = False)